In [1]:
import pandas as pd

import scipy as sc
from scipy.io import loadmat

import numpy as np

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

np.random.seed(123)

# Read Data

In [5]:
# splits
# images
# distances matrix
DATA_DIR = "../data"

# DATASET1: 17 Category Flowers
# 'val1', 'val2', 'val3', 'trn1', 'trn2', 'trn3', 'tst3', 'tst2', 'tst1'
'''splits = loadmat("%s/cat_flower/datasplits.mat" %DATA_DIR)

# 'D_siftbdy', 'D_siftint', 'D_hsv', 'D_hog',
distances = loadmat("%s/cat_flower/distancematrices17itfeat08.mat" %DATA_DIR)

imlist = loadmat("%s/cat_flower/trimaps/imlist.mat" %DATA_DIR)
'''

# DATASET2: 102 Category Flowers
# 'val1', 'val2', 'val3', 'trn1', 'trn2', 'trn3', 'tst3', 'tst2', 'tst1'
ds2_splits = loadmat("%s/102_cat_flower/setid.mat" %DATA_DIR)

# 'D_siftbdy', 'D_siftint', 'D_hsv', 'D_hog',
ds2_distances = loadmat("%s/102_cat_flower/distancematrices102.mat" %DATA_DIR)

ds2_labels = loadmat("%s/102_cat_flower/imagelabels.mat" %DATA_DIR)

# Prepare

In [2]:
'''KERNELS = {
    'gaussian': get_gaussian_kernel,
    'poly': get_ploynomial_kernel,
    'dist': get_distances_to_kernel
}'''

def get_gaussian_kernel(X, width=.2):
    return rbf_kernel(X, gamma=1.0/width)

def get_ploynomial_kernel(X, deg=2):
    return polynomial_kernel(X, degree=deg)

def get_distances_to_kernel(X):
    return (-X/X.mean())

def get_kernels(X, poly=False, gauss=False, distk=False):
    kernels = []
    
    if poly:
        degrees = [1,2,3]
        
        for deg in degrees:
            kernels.append(get_ploynomial_kernel(X, deg=deg))
            
            
    if gauss:
        widths = map(lambda x: 2**x, np.arange(-3,6))
        
        for w in widths:
            kernels.append(get_gaussian_kernel(X, width=w))
            
            
    if distk:
        kernels.append(get_distances_to_kernel(X))
        
    return kernels
    
    

In [3]:
# Helper Functions
log = np.log
gamma = sc.special.gamma
digamma = sc.special.digamma
det = np.linalg.det
diag = np.diag
outer = np.outer
tr = np.trace
dot =  np.dot
concat = np.concatenate
normal = np.random.normal
arr = np.array
cstack = np.column_stack
sqrt = np.sqrt
inv = np.linalg.inv
norm = sc.stats.norm

# Dummy

In [7]:
#x1 = np.random.normal(0,1,100)
x1 = np.random.multivariate_normal(mean=[0,0], cov=[[1,.5],[.5,1]], size=100)
y1 = np.repeat(1,100)
#x2 = np.random.normal(50,1,100)
x2 = np.random.multivariate_normal(mean=[20,20], cov=[[1,.5],[.5,1]], size=100)
y2 = np.repeat(-1,100)

N = 200
x = np.concatenate((x1,x2))
y = np.concatenate((y1,y2))

#K = arr([get_ploynomial_kernel(x.reshape((-1,1)), i+1) for i in range(3)])
K = arr([get_ploynomial_kernel(x, i+1) for i in range(3)])
print K[0].shape

P = 3

(200, 200)


In [8]:
x.shape

(200, 2)

# BEMKL

<font size=4> $\lambda_i \sim Gamma(\alpha_\lambda,\beta_\lambda)$ <br>
    $E\_a2_i = E(a_i^2) = Var(a_i) + \mu_{a_i}^2$ <br>
    $Corr\_a = E(aa^T) = \Sigma_a + \mu_{a}\mu_{a}^T$ <br>
    $corr\_g_i = E(g_ig_i^T) = \Sigma_{g_i} + \mu_{g_i}\mu_{g_i}^T$ <br>
    $corr_G$ <br>
     <br>
</font>

In [16]:
# Hyper Parameters (prior)
sparse = False

alpha_lambda, beta_lambda = 1.0,1.0
alpha_gamma, beta_gamma = 1.0,1.0
alpha_omega, beta_omega = 1.0,1.0

if sparse:
    alpha_omega, beta_omega = 10.0**-10, 10.0**10

rv_lambda = np.random.gamma(alpha_lambda, beta_lambda, N) 
rv_a = np.array([np.random.normal(0, 1.0/rv_lambda[i], 1) for i in range(N)])

rv_gamma = np.random.gamma(alpha_gamma, beta_gamma, 1) 
rv_b = np.random.normal(0, 1.0/rv_gamma, 1)

rv_omega = np.random.gamma(alpha_omega, beta_omega, P) 
rv_e = np.array([np.random.normal(0, 1.0/rv_omega[i], 1) for i in range(P)])

# The conditionals parameters
p_alpha_lambda = np.repeat(alpha_lambda, N)
p_beta_lambda = np.repeat(beta_lambda, N)
p_alpha_gamma = alpha_gamma
p_beta_gamma = beta_gamma
p_alpha_omega = np.repeat(alpha_omega, P)
p_beta_omega = np.repeat(beta_omega, P)

mu_a = np.repeat(0, N)
mu_b = 0
mu_e = np.repeat(0, P)

#cov_a = diag(1.0/rv_lambda)
cov_a = diag(np.repeat((alpha_lambda/beta_lambda**2)**-1, N))
print 'cov_a',cov_a[:5,:5]
#var_b = rv_gamma**-1
var_b = (alpha_gamma/beta_gamma**2)**-1
#cov_e = np.diag(1.0 / rv_omega)
cov_e = diag(np.repeat((alpha_omega/beta_omega**2)**-1, P))

#mu_g = np.vstack([dot(a.T, K[i]) for i in range(P)])

#mu_g = arr([dot(rv_a.T,K[j,...]) for j in range(P)]).squeeze()
mu_g = arr([dot(mu_a.T,K[j,...]) for j in range(P)]).squeeze()
#mu_g = np.zeros((P,N))
print 'mu_g',mu_g.shape, mu_g[:,:5]

#G = np.array([[np.random.normal(mu_g[i,j],1, 1)[0] for j in range(N)] for i in range(P)])
#print 'G', G.shape
'''cov_g = arr([(outer(G[:,i],G[:,i])- outer(mu_g[:,i], mu_g[:,i])) 
                  for i in range(N)])
'''
cov_g = np.zeros((N,P,P))
cov_g[:,:,:] = np.eye(P)
print 'cov_g',cov_g[:5,...]

mu_b_e = np.concatenate(([mu_b],mu_e))
#cov_b_e = np.vstack(([rv_b if i==0 else 0 for i in range(P+1)],
#                np.hstack((np.zeros((P,1)),cov_e))))
cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
print 'COV_b_e', cov_b_e.shape

omega = np.random.gamma(alpha_omega, 1.0/beta_omega, P)
e = concat([normal(0,omega[i],1) for i in range(P)])
print 'e', e.shape
#mu_f = np.array([dot(e.T,G[:,i])+ rv_b for i in range(N)])
#mu_f = dot(e.T,mu_g)+ mu_b 
mu_f = dot(mu_b_e[1:], mu_g) + mu_b
print 'mu_f',mu_f.shape

E_a2 = np.diag(cov_a) + mu_a**2 
E_b2 = 1.0 / rv_gamma + mu_b**2
E_e2 = np.diag(cov_e) + mu_e**2

corr_a = cov_a + outer(mu_a, mu_a)
corr_e = cov_b_e[1:,1:] + outer(mu_b_e[1:], mu_b_e[1:])
corr_g = cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])

# inter-row covariance
'''cov_G = diag(cstack([diag(cov_g[i,...]) for i in range(N)]).sum(axis=1))
corr_G = cov_G + dot(mu_g,mu_g.T)'''
corr_G = np.zeros(P)

'''for i in range(P):
    for j in range(P):
        if i==j:
            continue
        corr_G[i,j] = dot(dot(K[i,...].T,corr_a),K[j,...])'''

thresh = 1.0 * 10**-4
ELBO_init = 0

K2 = np.sum([dot(K[i,...], K[i,...].T) for i in range(P)], axis=0)

cov_a [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
mu_g (3, 200) [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
cov_g [[[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]]
COV_b_e (4, 4)
e (3,)
mu_f (200,)


### Tricks:
- for inverting the matrix add $\epsilon I$ to the matrix

In [ ]:
it = 1

while True: 
    # Update the parameters:
    ########################
    E_a2 = diag(cov_a) + mu_a**2
    corr_a = cov_a + outer(mu_a, mu_a)
    
    p_alpha_lambda = np.repeat(alpha_lambda + 0.5, N)
    p_beta_lambda = (beta_lambda**-1 + E_a2/2.0)**-1
    E_lambda = p_alpha_lambda * p_beta_lambda
    

    #mu_a = dot(cov_a, np.sum([dot(K[i], mu_g[i,:]) for i in range(P)]))
    print mu_g.shape
    z = np.sum([dot(K[i,:,:], mu_g[i,:]) for i in range(P)], axis=0)
    mu_a = dot(cov_a, z)
    cov_a = inv(diag(E_lambda) + K2)
    print 'cov_a', cov_a[:5,:5]
    
    print mu_b_e.shape, mu_e.shape
    # G: PxN
    # mu_g: PxN
    #mu_g = cstack([dot(cov_g[i],(dot(K[:,i,:], mu_a) + mu_f[i]* mu_e + mu_b_e))
    #            for i in range(N)])
    z = dot(cov_b_e[0,1:],inv(cov_b_e[1:,1:]))
    E_be = mu_b_e[0] * mu_b_e[1:] + dot(cov_b_e[1:,1:], z)
    print 'E_be', E_be
    #z = dot(K, mu_a)
    #print 'Ka', z[:5,:5]
    print 'cov_g', cov_g[:5,:5]
    mu_g = cstack([dot(cov_g[i,:,:], (dot(K[:,i,:], mu_a) + mu_f[i]* mu_b_e[1:] + E_be)) for i in range(N)])
    print 'mu_g',mu_g[:,:5]
    #cov_g = (np.identity(P) + cov_e + np.outer(mu_e,mu_e))**-1
    cov_g[:,:,:] = inv(np.eye(P) + corr_e)
    print 'cov_g', cov_g[:5,...]
    corr_g = cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])
    cov_G = diag(cstack([diag(cov_g[i,...]) for i in range(N)]).sum(axis=1))
    corr_G = cov_G + dot(mu_g,mu_g.T)
    
    E_b2 = cov_b_e[0,0] + mu_b_e[0]**2
    p_alpha_gamma = alpha_gamma + 0.5
    p_beta_gamma = (beta_gamma**-1 + E_b2/2.0)**-1
    
    E_e2 = diag(cov_b_e[1:,1:]) + mu_b_e[1:]**2
    corr_e = cov_b_e[1:,1:] + outer(mu_b_e[1:], mu_b_e[1:])
    print 'corr_e', corr_e
    
    p_alpha_omega = np.repeat(alpha_omega + 0.5, P)
    p_beta_omega = (beta_omega**-1 + E_e2/2.0)**-1
    
    E_gamma = p_alpha_gamma * p_beta_gamma
    E_omega = p_alpha_omega * p_beta_omega
    
    print mu_f.shape
    z = concat(([mu_f.sum()], dot(mu_g,mu_f).squeeze()))
    #mu_b_e = dot(cov_b_e, [mu_f.sum() if i==0 else dot(mu_g,mu_f) for i in range(P+1)])
    mu_b_e = dot(cov_b_e, z)
    cov_b_e[0,0] = E_gamma + N
    cov_b_e[1:,0] = mu_g.sum(axis=1)
    cov_b_e[0,1:] = cov_b_e[1:,0]
    #cov_b_e[1:,1:] = diag(E_omega) + dot(G,G.T)
    cov_b_e[1:,1:] = diag(E_omega) + corr_G
    cov_b_e = inv(cov_b_e)
    
    v = 1
    LU = arr(map(lambda x: (-10**5, -v) if x==-1 else (v,10**5), y))
    eg = dot(mu_b_e[1:].T, mu_g)
    alpha_f = LU[:,0] - eg - mu_b_e[0]
    beta_f = LU[:,1] - eg - mu_b_e[0]
    pdf_alpha_f = norm.pdf(alpha_f)
    pdf_beta_f = norm.pdf(beta_f)
    Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
    mu_f = eg + mu_b_e[0] + (pdf_alpha_f - pdf_beta_f)/Z
    var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 
    E_f2 = var_f + mu_f**2
    
    # Calculate the ELBO
    ###################
    # E(log(p(lambda)))    
    #E_log_lambda = np.array([log(np.random.gamma(p_alpha_lambda[i], p_beta_lambda[i], 100)).mean()
    #                         for i in range(N)])
    E_log_lambda = digamma(p_alpha_lambda) - log(p_beta_lambda)
    E_lp_lambda = ((alpha_lambda - 1) * E_log_lambda - 1.0/beta_lambda * E_lambda - log(gamma(alpha_lambda))
            - alpha_lambda * log(beta_lambda)).sum(axis=0)
    print 'lp_lam', E_lp_lambda

    # E(log(p(a|lambda)))
    E_lp_a_lambda = ( - 0.05 * tr(dot(diag(E_lambda), cov_a)).sum() - 0.5 * N *log(2.0*np.pi) 
                     + 0.05 * log(np.prod(E_lambda)))
    print 'lp_a_lam', E_lp_a_lambda

    # E(log(p(G|a,K)))
    #k_i = np.array([K[j][i,:] for j in range(P)])
    '''E_lp_G = np.sum([-.5 * (diag(cov_g[i])+mu_g[:,i]**2).sum() 
                     + np.trace(dot(k_i.T, dot(mu_g[:,i],mu_a)))
                     - 0.5 * np.trace( (k_i**2).sum() * (cov_a + outer(mu_a,mu_a) )) 
                     for i in range(N)] )'''
    E_g2 = arr([(diag(cov_g[i,...]) + mu_g[:,i]**2).sum() for i in range(N)])
    #print 'eg2',E_g2.shape
    kga = arr([dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)])
    #print 'kga', kga[0,:5,:5]
    
    #print 'corra', corr_a.shape
    kaa = arr([dot(K[:,:,i].T , dot(K[:,:,i],corr_a)) for i in range(N)])
    #print 'kaa', kaa[0,:5,:5]

    E_lp_G = np.sum([-.5 * E_g2[i] + tr(kga[i,...]) - 0.5 * tr(kaa[i,...]) 
                     - 0.5 * P *log(2.0*np.pi) for i in range(N)])
    print 'lp_g', E_lp_G

    # E(log(p(gamma)))
    E_log_gamma = digamma(p_alpha_gamma) - log(p_beta_gamma)
    E_lp_gamma = ((alpha_gamma - 1) * E_log_gamma - E_gamma/beta_gamma - log(gamma(alpha_gamma)) 
                  - alpha_gamma * log(beta_gamma))
    print 'lp_gamma', E_lp_gamma

    # E(log(p(b|gamma)))
    E_lp_b = (- 0.5 * E_gamma * E_b2 - 0.5 *log(2.0*np.pi) + 0.5 * log(E_gamma))
    print 'lp_b', E_lp_b

    # E(log(p(omega)))
    E_log_omega = digamma(p_alpha_omega) - log(p_beta_omega)
    E_lp_omega = ((alpha_omega - 1) * E_log_omega - E_omega/beta_omega - log(gamma(alpha_omega))
                  - alpha_omega * log(beta_omega) ).sum()
    print 'lp_omega', E_lp_omega

    # E(log(p(e|omega)))
    E_lp_e = ( -0.5 * tr(dot(diag(E_omega), corr_e)) - 0.5 * P * log(2.0*np.pi)
             + 0.05 * log(np.prod(E_omega)))
    print 'lp_e', E_lp_e

    # E(log(p(f|e,b,G)))
    '''E_lp_f = (-0.5 * E_f2 + (dot(mu_e.T,mu_g) + b) * mu_f 
              - 0.5 * ([dot(Cov_e,Cov_g).diag.sum() for i in range(N)]
            + 2 * dot(E_b_e, E_G) + E_b2) - 0.5* log(2.0*np.pi)).sum()'''
    E_lp_f = (-0.5 * E_f2 + (dot(mu_b_e[1:].T,mu_g) + mu_b_e[0]) * mu_f 
              - 0.5 * arr([(tr(dot(corr_e,corr_g[i,...])) + 2 * dot(E_be.T, mu_g[:,i]) + E_b2) 
                           for i in range(N)]) 
              - 0.5* log(2.0*np.pi)).sum()
    print 'lp_f', E_lp_f


    # E(log(q(lambda)))
    E_lq_lambda = (-p_alpha_lambda - log(p_beta_lambda) - log(gamma(p_alpha_lambda)) 
                   - (1-p_alpha_lambda) * digamma(p_alpha_lambda)).sum()
    print 'lq_lam', E_lq_lambda

    # E(log(q(a)))
    det_cov_a = det(cov_a)
    E_lq_a = -0.5 * N * (log(2.0*np.pi)+1) - 0.5 * log(det_cov_a) if det_cov_a > 0 else 10**-6
    print 'lq_a', E_lq_a

    # E(log(q(G)))
    E_lq_G = (-0.5 * P *(log(2.0*np.pi)+1) - 0.5 * np.array([log(det(cov_g[i])) for i in range(N)])).sum()
    print 'lq_g', E_lq_G

    # E(log(q(gamma)))
    E_lq_gamma = (-p_alpha_gamma - log(p_beta_gamma) - log(gamma(p_alpha_gamma)) 
                   - (1-p_alpha_gamma) * digamma(p_alpha_gamma))
    print 'lq_gamma', E_lq_gamma

    # E(log(q(omega)))
    E_lq_omega = (-p_alpha_omega - log(p_beta_omega) - log(gamma(p_alpha_omega)) 
                   - (1-p_alpha_omega) * digamma(p_alpha_omega)).sum()
    print 'lq_omega', E_lq_omega

    # E(log(q(b,e)))
    det_cov_b_e = det(cov_b_e)
    E_lq_b_e = (-0.5 * (P+1) *(log(2.0*np.pi)+1) - 0.5 * log(det_cov_b_e) if det_cov_b_e > 0 else 10**-6 )
    print 'lq_be', E_lq_b_e

    # E(log(q(f)))
    E_lq_f = (-0.5 * (log(2.0*np.pi) + var_f) - log(Z)).sum()
    print 'lq_f', E_lq_f


    ELBO = (E_lp_lambda + E_lp_a_lambda + E_lp_G + E_lp_gamma + E_lp_b + E_lp_omega + E_lp_e + E_lp_f
            - E_lq_lambda - E_lq_a - E_lq_G - E_lq_gamma - E_lq_omega - E_lq_b_e - E_lq_f)
    
    print "ITER %d: %f" %(it, ELBO)
    
    if np.abs(ELBO - ELBO_init) < thresh:
        print 'Convergence'
        break
        
    ELBO_init = ELBO
    
    if it == 500:
        break
        
    it += 1

    
    

In [18]:
cov_a

array([[ 8.94765797e-01,  1.53997113e-02, -6.68839965e-02, ...,
         3.51495315e-04,  8.35799495e-04, -3.77363252e-04],
       [ 1.45898582e-02,  9.31361402e-01,  2.10781179e-03, ...,
         4.77476035e-04,  3.41326664e-04, -5.52339457e-05],
       [-7.18667743e-02,  1.53023603e-04,  8.16989770e-01, ...,
         1.04663018e-03, -1.26781897e-04,  1.44923825e-04],
       ...,
       [ 2.16164005e-04, -1.74761597e-04, -1.48339394e-03, ...,
        -7.01513476e-04, -2.52982518e-04,  2.17458091e-04],
       [-2.49656075e-03,  1.58891341e-03, -7.58336915e-03, ...,
         8.48814872e-04, -7.35621304e-04,  3.95359777e-05],
       [-1.46755161e-03, -1.37242484e-04, -9.54142798e-03, ...,
         4.80753069e-04, -5.41864664e-04,  4.72814889e-05]])

# Test

In [296]:
rv_lambda = np.random.gamma(p_alpha_lambda, p_beta_lambda, N) 
rv_a = np.array([np.random.normal(0, sqrt(1.0/rv_lambda[i]), 1) for i in range(N)])

rv_gamma = np.random.gamma(p_alpha_gamma, p_beta_gamma, 1) 
rv_b = np.random.normal(0, sqrt(1.0/rv_gamma), 1)

rv_omega = np.random.gamma(p_alpha_omega, p_beta_omega, P) 
rv_e = np.array([np.random.normal(0, sqrt(1.0/rv_omega[i]), 1) for i in range(P)])


In [317]:
rv_a = np.random.multivariate_normal(mu_a, cov_a, 1).squeeze()
print rv_a
b_e = np.random.multivariate_normal(mu_b_e, cov_b_e, 1).squeeze()
#print b_e
rv_b, rv_e = b_e[0], b_e[1:]
print rv_b,rv_e

f = dot(rv_a.T, np.sum([rv_e[j] * K[j,...] for j in range(P)], axis=0))+ rv_b
f

[-4.22340214e-01 -1.95669137e-01  3.74685363e-01 -3.50196399e-01
 -7.41773559e-01  8.91851018e-01 -3.55322345e-01 -6.09156799e-01
 -2.97242559e-02 -4.47785156e-01  8.75423590e-01 -2.06346488e-02
  9.37365382e-01 -1.11086212e+00 -2.24219002e-01  3.33471602e-01
  4.23004387e-01 -1.03389607e+00  1.85520931e-01  7.06933478e-01
  7.16784085e-01 -9.70574741e-01 -3.47814262e-01  2.08926987e+00
  5.74450901e-01  3.90188286e-01 -2.23533682e-02 -4.39450497e-01
 -8.18921128e-01 -1.77419513e-01 -6.57282717e-01  1.52352620e-01
 -8.23659850e-01  4.19176526e-01  6.30489162e-02  3.23075234e-01
  1.19547154e-01 -1.52709683e+00 -4.70300710e-01  3.43879484e-01
 -1.56498285e-01 -8.64096399e-01  4.68170403e-03  7.63003964e-01
 -2.38618627e-01 -1.02836189e+00  1.87321706e-01  3.96037489e-01
 -1.59689681e+00  2.63777221e-01 -4.79365943e-01  3.41991851e-01
  8.76715255e-02 -6.10827046e-02 -7.86012745e-01  2.23692252e-02
 -7.14414299e-02  7.54542691e-01  1.02805694e+00  1.42805659e-01
  4.96165790e-01  4.51668

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  """Entry point for launching an IPython kernel.


array([0.06202674, 0.06202673, 0.06202673, 0.06202673, 0.06202673,
       0.06202674, 0.06202672, 0.06202674, 0.06202672, 0.06202674,
       0.06202672, 0.06202674, 0.06202675, 0.06202673, 0.06202675,
       0.06202673, 0.06202674, 0.06202672, 0.06202674, 0.06202673,
       0.06202673, 0.06202675, 0.06202672, 0.06202673, 0.06202674,
       0.06202674, 0.06202671, 0.06202674, 0.06202672, 0.06202673,
       0.06202673, 0.0620267 , 0.06202673, 0.06202674, 0.06202673,
       0.06202674, 0.06202676, 0.06202675, 0.06202673, 0.06202677,
       0.06202673, 0.06202674, 0.06202671, 0.06202674, 0.06202673,
       0.06202673, 0.06202674, 0.06202674, 0.06202672, 0.06202673,
       0.06202673, 0.06202672, 0.06202674, 0.06202676, 0.06202674,
       0.06202673, 0.06202671, 0.06202672, 0.06202674, 0.06202675,
       0.06202673, 0.06202672, 0.06202672, 0.06202673, 0.06202671,
       0.06202673, 0.06202674, 0.06202671, 0.06202673, 0.06202671,
       0.06202672, 0.06202673, 0.06202675, 0.06202674, 0.06202

In [298]:
f = dot(rv_a.T, np.sum([rv_e[j] * K[j,...] for j in range(P)], axis=0))+ rv_b
f

array([[ 1.12638506e+06, -4.61826208e+05, -4.05379471e+05,
        -1.85042912e+04, -5.16563565e+05,  4.85996868e+05,
        -5.28950160e+06,  1.46638528e+06, -9.73502635e+05,
         1.50544557e+05, -3.80025758e+06,  2.95028822e+05,
         3.21761949e+06,  6.66316482e+01,  9.22900338e+06,
        -4.07924555e+03,  1.92081602e+05, -1.77277562e+06,
         2.30534277e+05, -3.93058569e+05, -8.20253938e+05,
         9.79469487e+06, -4.66761681e+06, -2.00434846e+03,
         9.41173388e+05,  3.92750490e+04, -1.45768211e+07,
         5.80965501e+05, -5.99532381e+06, -1.80874103e+05,
        -2.68397692e+04, -5.82750348e+07, -1.48857268e+03,
         7.71777273e+05, -8.02252465e+03,  2.06170536e+04,
         2.84002645e+07,  3.59358281e+06,  1.20234811e+03,
         4.23555136e+07, -5.58964447e+05,  3.76049011e+05,
        -9.01022527e+06,  1.75189722e+05, -1.79702931e+03,
         1.26727746e+03,  1.42493107e+05,  6.46433415e+04,
        -2.78554696e+06, -1.14387765e+05,  4.59461238e+0

In [305]:
from sklearn.metrics import accuracy_score, f1_score

pred = np.sign(f).squeeze()
print 'Accuracy:', accuracy_score(y, pred)
print 'F1 score:', f1_score(y, pred)

Accuracy: 0.255
F1 score: 0.4063745019920319


In [308]:
y,f

(array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 array([[ 1.12638506e+06, -4.61826208e+05, -4.05379471e+05,
         -1.85042912e+04, -5.16563565e+05,

In [295]:
p_alpha_lambda ,  p_beta_lambda ,  p_alpha_gamma ,  p_beta_gamma , p_alpha_omega ,    p_beta_omega 
print

In [141]:
#n[K[j][i,:] for j in range(P)]
print mu_g.shape
#np.sum((dot(K[i,:,:], mu_g[i,:]) for i in range(P))).shape
#K[:,0,:].shape
#z.shape,z[:,1].shape,  (mu_f[1]* mu_e).shape,  E_be.shape
#arr([(diag(cov_g[i,...]) + mu_g[:,i]).sum() for i in range(N)]).shape
#dot(K[:,:,0].T , outer(mu_g[:,0],mu_a)).shape

#[dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)]
#(cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])).shape

from scipy.stats import norm

v = 1
LU = arr(map(lambda x: (-np.inf, -v) if x==-1 else (v,np.inf), y))
eg = dot(mu_e.T, mu_g)
alpha_f = LU[:,0] - eg - mu_b
beta_f = LU[:,1] - eg - mu_b
pdf_alpha_f = norm.pdf(alpha_f)
pdf_beta_f = norm.pdf(beta_f)
Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
mu_f = eg + mu_b + (pdf_alpha_f - pdf_beta_f)/Z
var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 

(3, 200)


/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [137]:
dot(mu_e.T, mu_g) - mu_b
from scipy.stats import norm
norm.cdf([np.inf,2])

array([1.        , 0.97724987])

In [147]:
E_g2 = arr([(diag(cov_g[i,...]) + mu_g[:,i]**2).sum() for i in range(N)])
#print 'eg2',E_g2.shape
kga = arr([dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)])
#print 'kga', kga.shape

#print 'corra', corr_a.shape
kaa = arr([dot(K[:,:,i].T , dot(K[:,:,i],corr_a)) for i in range(N)])
#print 'kaa', kaa.shape

E_lp_G = np.sum([-.5 * E_g2[i] + tr(kga[i,...]) - 0.5 * tr(kaa[i,...]) 
                 - 0.5 * P *log(2.0*np.pi) for i in range(N)])
print 'lp_g', E_lp_G

ValueError: all the input arrays must have same number of dimensions

In [188]:
z = np.sum([dot(K[i,:,:], mu_g[i,:]) for i in range(P)], axis=0)
print z
mu_a = dot(cov_a, z)
print mu_a
cov_a = diag(E_lambda) + K2

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan 

In [208]:
inv = np.linalg.inv

cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
print cov_b_e
z = dot(cov_b_e[0,1:],inv(cov_e))
print z[:3]
E_be = mu_b * mu_e + dot(cov_e, z)
print 'E_be', E_be

z = dot(K, mu_a)
print z[:5,:5]
mu_g = cstack([dot(cov_g[i,:,:], (z[:,i] + mu_f[i]* mu_e + E_be)) for i in range(N)])
mu_g

[[0.60913865 0.         0.         0.        ]
 [0.         0.46786111 0.         0.        ]
 [0.         0.         0.6651148  0.        ]
 [0.         0.         0.         0.59561409]]
[0. 0. 0.]
E_be [0. 0. 0.]
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan

In [225]:
v = 1
LU = arr(map(lambda x: (-10**5, -v) if x==-1 else (v,10**5), y))
print LU[:5]
eg = dot(mu_e.T, mu_g)
alpha_f = LU[:,0] - eg - mu_b
beta_f = LU[:,1] - eg - mu_b
print alpha_f[:4],beta_f[:4]
pdf_alpha_f = norm.pdf(alpha_f)
pdf_beta_f = norm.pdf(beta_f)
print pdf_alpha_f[:4],pdf_beta_f[:4]
Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
print Z[:4]
mu_f = eg + mu_b + (pdf_alpha_f - pdf_beta_f)/Z
var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 
print var_f[:5]
E_f2 = var_f + mu_f**2

E_lp_f = (-0.5 * E_f2 + (dot(mu_e.T,mu_g) + mu_b) * mu_f 
              - 0.5 * arr([(tr(dot(corr_e,corr_g[i,...])) + 2 * dot(E_be.T, mu_g[:,i]) + E_b2) 
                           for i in range(N)]) 
              - 0.5* log(2.0*np.pi)).sum()

[[     1 100000]
 [     1 100000]
 [     1 100000]
 [     1 100000]
 [     1 100000]]
[1. 1. 1. 1.] [100000. 100000. 100000. 100000.]
[0.24197072 0.24197072 0.24197072 0.24197072] [0. 0. 0. 0.]
[0.15865525 0.15865525 0.15865525 0.15865525]
[0.19909767 0.19909767 0.19909767 0.19909767 0.19909767]


In [267]:
#z = dot(cov_b_e[0,1:],inv(cov_e))
#E_be = mu_b * mu_e + dot(cov_e, z)
print 'E_be', E_be
#z = dot(K, mu_a)
#print 'Ka', z[:5,:5]
print 'cov_g', cov_g[:5,:5]
mu_g = cstack([dot(cov_g[i,:,:], (dot(K[:,i,:], mu_a) + mu_f[i]* mu_e + E_be)) for i in range(N)])

array([[-3.21174770e-89, -9.51330485e-50,  7.41605516e-53,
        -1.46269847e-56],
       [-9.51330485e-50,  1.08494586e-13,  2.58343028e-15,
        -1.03037943e-18],
       [ 7.41605516e-53,  2.58343028e-15,  6.15156408e-17,
        -2.45352309e-20],
       [-1.46269847e-56, -1.03037943e-18, -2.45352309e-20,
         9.78886525e-24]])

In [260]:
log(det(inv(diag(E_lambda) + K2))) if det(inv(diag(E_lambda) + K2))!=0 else 10**-6# + np.random.normal(scale=10**-3,size=cov_a.shape))


1e-06

In [281]:
mu_a

array([ 3.94455581e+47,  1.27025937e+49, -6.57706512e+49,  7.95515751e+49,
        2.21548718e+49,  4.07051042e+48, -1.03261371e+48, -3.41738849e+48,
       -2.89784803e+48, -1.66023250e+49,  1.05188155e+48, -6.94712814e+48,
       -8.86142411e+47,  4.21576855e+49,  3.00168030e+47, -2.65004215e+48,
       -2.13687525e+49,  1.12230730e+48, -8.03866042e+48,  4.56567617e+48,
       -1.15294853e+48, -5.50694602e+47,  1.19540915e+48, -6.67059890e+48,
       -1.98530154e+48,  9.35521613e+49,  2.59759080e+47,  3.72636532e+48,
        1.63277136e+48,  1.10361036e+49, -1.47028813e+48, -1.40583898e+47,
       -7.24310363e+48,  1.57318377e+48, -3.16550158e+48, -3.23012914e+48,
        9.70528431e+46, -8.01845594e+47, -1.51368538e+48,  1.07400389e+47,
        5.85555127e+48, -5.87919006e+48, -1.03921473e+46,  7.32311871e+48,
       -3.87479908e+48, -5.27439385e+48, -4.31858391e+48, -4.43429802e+49,
        8.21202641e+47, -6.44562899e+48, -3.41754460e+48,  3.23271602e+47,
        4.51875540e+48, -

In [207]:
cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
cov_b_e

array([[0.60913865, 0.        , 0.        , 0.        ],
       [0.        , 0.46786111, 0.        , 0.        ],
       [0.        , 0.        , 0.6651148 , 0.        ],
       [0.        , 0.        , 0.        , 0.59561409]])

In [257]:
np.linalg.matrix_rank(inv(diag(E_lambda) + K2))

198

In [278]:
dot(K[:,:,1].T, outer(mu_g[:,1], mu_a.T)) 

array([[-3.89878831e+095, -1.25552093e+097,  6.50075339e+097, ...,
        -1.36040446e+091,  1.23986229e+091, -3.04799480e+091],
       [-2.38446233e+096, -7.67864812e+097,  3.97579975e+098, ...,
        -8.32010596e+091,  7.58288138e+091, -1.86412501e+092],
       [-2.32630335e+096, -7.49135964e+097,  3.87882675e+098, ...,
        -8.11717180e+091,  7.39792873e+091, -1.81865748e+092],
       ...,
       [ 1.03223265e+100,  3.32408327e+101, -1.72112190e+102, ...,
         3.60177008e+095, -3.28262589e+095,  8.06978865e+095],
       [ 1.03804905e+100,  3.34281372e+101, -1.73082002e+102, ...,
         3.62206522e+095, -3.30112274e+095,  8.11526006e+095],
       [ 1.05695208e+100,  3.40368688e+101, -1.76233853e+102, ...,
         3.68802360e+095, -3.36123670e+095,  8.26304022e+095]])

In [280]:
mu_g[:,1]

array([-8.60894603e+47, -1.12326416e+48, -1.11950883e+48])

In [284]:
t = np.zeros((3,2,2))
t[:,:,:] = np.eye(2)
print t

[[[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]]


In [326]:
np.diag?